In [ ]:
Escreva uma consulta para responder às seguintes questões:
Quais os produtos foram vendidos entre os dias 01/01/2020 e 31/07/2020? 
Qual forma de pagamento mais utilizada?
Qual forma de pagamento gerou mais receita?
Qual vendedor concretizou mais vendas em cada mês de 2019?
Qual a quantidade média de itens por pedido em cada mês de 2020?
Qual categoria mais vendida em cada ano?
Qual categoria possui mais produtos?
Quais os 3 itens mais vendidos em cada mês?
Quais os 3 meses com maior valor de venda em 2020?
Quais os 5 vendedores receberam mais comissão em 2019?
Qual o produto mais vendido nos meses de julho e dezembro?

In [ ]:
--Quais os produtos foram vendidos entre os dias 01/01/2020 e 31/07/2020? 

SELECT 
	produto.cd_produto, 
	produto.desc_produto
FROM pedido
LEFT JOIN item_pedido ON pedido.nr_pedido = item_pedido.nr_pedido
LEFT JOIN produto ON produto.cd_produto = item_pedido.cd_produto
WHERE TO_CHAR(pedido.dt_pedido, 'YYYY-MM-DD') > '2020-01-01' AND TO_CHAR(pedido.dt_pedido, 'YYYY-MM-DD') < '2020-07-31'

In [ ]:
--Qual forma de pagamento mais utilizadas

SELECT
	pedido.cd_forma_pag,
	formas_pagamento.desc_forma_pag,
	COUNT(pedido.cd_forma_pag)
FROM pedido
LEFT JOIN formas_pagamento ON formas_pagamento.cd_forma_pag = pedido.cd_forma_pag
GROUP BY pedido.cd_forma_pag, formas_pagamento.desc_forma_pag
HAVING COUNT(pedido.cd_forma_pag) = (SELECT 
							  	 		 MAX(COUNT(cd_forma_pag)) OVER () AS maximo
							  		 FROM pedido
									 GROUP BY cd_forma_pag
							 		 LIMIT 1);

In [ ]:
--Qual forma de pagamento gerou mais receita?

SELECT 
	pedido.cd_forma_pag,
	formas_pagamento.desc_forma_pag,
	SUM(item_pedido.preco_item * item_pedido.qnt_item_pedido) AS receita
FROM item_pedido
LEFT JOIN pedido ON item_pedido.nr_pedido = pedido.nr_pedido
LEFT JOIN formas_pagamento ON pedido.cd_forma_pag = formas_pagamento.cd_forma_pag
GROUP BY pedido.cd_forma_pag, formas_pagamento.desc_forma_pag
HAVING SUM(item_pedido.preco_item * item_pedido.qnt_item_pedido) = (SELECT 
																		MAX(SUM(item_pedido.preco_item * item_pedido.qnt_item_pedido)) OVER ()
																	FROM item_pedido
																	LEFT JOIN pedido ON item_pedido.nr_pedido = pedido.nr_pedido
																	GROUP BY pedido.cd_forma_pag
																	LIMIT 1)

In [ ]:
--Qual vendedor concretizou mais vendas em cada mês de 2019?

SELECT 
	pedido.cd_vendedor,
	vendedor.nm_vendedor,
	COUNT(pedido.cd_vendedor) AS qnt_vendas
FROM pedido
LEFT JOIN vendedor ON pedido.cd_vendedor = vendedor.cd_vendedor
GROUP BY pedido.cd_vendedor, nm_vendedor
HAVING COUNT(pedido.cd_vendedor) = (SELECT
							 			MAX(COUNT(cd_vendedor)) OVER () AS maximo
							 		FROM pedido
							 		GROUP BY cd_vendedor
							 		LIMIT 1)

In [ ]:
--Qual a quantidade média de itens por pedido em cada mês de 2020?

SELECT
	TO_CHAR(pedido.dt_pedido, 'YYYY-MM') AS mes,
	ROUND(AVG(item_pedido.qnt_item_pedido)::numeric, 2) AS media_qnt_pedidos
FROM item_pedido
LEFT JOIN pedido ON item_pedido.nr_pedido = pedido.nr_pedido
GROUP BY TO_CHAR(pedido.dt_pedido, 'YYYY-MM')
ORDER BY TO_CHAR(pedido.dt_pedido, 'YYYY-MM');

In [ ]:
-- Qual categoria mais vendida em cada ano?

CREATE OR REPLACE VIEW tabela_temporaria AS
(SELECT 
	TO_CHAR(pedido.dt_pedido, 'YYYY') AS ano,
	produto.cd_categoria,
	SUM(item_pedido.qnt_item_pedido) AS vendas,
	RANK() OVER (PARTITION BY TO_CHAR(pedido.dt_pedido, 'YYYY') ORDER BY SUM(item_pedido.qnt_item_pedido) DESC) AS ranque
FROM item_pedido
LEFT JOIN produto ON item_pedido.cd_produto = produto.cd_produto
LEFT JOIN pedido ON item_pedido.nr_pedido = pedido.nr_pedido
GROUP BY  
	TO_CHAR(pedido.dt_pedido, 'YYYY'),
	produto.cd_categoria);

SELECT
	ano,
	tabela_temporaria.cd_categoria,
	categoria.nm_categoria,
	vendas
FROM tabela_temporaria
LEFT JOIN categoria ON tabela_temporaria.cd_categoria = categoria.cd_categoria
WHERE ranque = 1;

In [ ]:
-- Qual categoria possui mais produtos?

CREATE OR REPLACE VIEW tabela_temporaria AS
(SELECT 
	cd_categoria,
	COUNT(cd_produto) AS quantidade_de_itens,
	RANK () OVER (ORDER BY COUNT(cd_produto) DESC) AS ranque
FROM produto
GROUP BY cd_categoria
ORDER BY cd_categoria);

SELECT
	tabela_temporaria.cd_categoria,
	categoria.nm_categoria,
	quantidade_de_itens
FROM tabela_temporaria
LEFT JOIN categoria ON tabela_temporaria.cd_categoria = categoria.cd_categoria
WHERE ranque = 1

In [ ]:
-- Quais os 3 itens mais vendidos em cada mês?

CREATE OR REPLACE VIEW tabela_temporaria AS
(SELECT 
	TO_CHAR(pedido.dt_pedido, 'YYYY-MM') AS mes,
	item_pedido.cd_produto,
	SUM(item_pedido.qnt_item_pedido) AS qnt_item_vendido,
	DENSE_RANK() OVER (PARTITION BY TO_CHAR(pedido.dt_pedido, 'YYYY-MM') ORDER BY SUM(item_pedido.qnt_item_pedido) DESC) AS ranque
FROM item_pedido
LEFT JOIN pedido ON item_pedido.nr_pedido = pedido.nr_pedido
GROUP BY 
	TO_CHAR(pedido.dt_pedido, 'YYYY-MM'), 
	item_pedido.cd_produto);
	
SELECT 
	tabela_temporaria.mes,
	tabela_temporaria.cd_produto,
	produto.desc_produto,
	tabela_temporaria.qnt_item_vendido	
FROM tabela_temporaria
LEFT JOIN produto ON tabela_temporaria.cd_produto = produto.cd_produto
WHERE ranque <= 3;

In [ ]:
-- Quais os 3 meses com maior valor de venda em 2020?

CREATE OR REPLACE VIEW tabela_temporaria AS
(SELECT 
	TO_CHAR(pedido.dt_pedido, 'YYYY-MM') AS mes,
	SUM(item_pedido.qnt_item_pedido * item_pedido.preco_item) AS valor_de_vendas,
	RANK () OVER (ORDER BY SUM(item_pedido.qnt_item_pedido) DESC) AS ranque
FROM item_pedido
LEFT JOIN pedido ON item_pedido.nr_pedido = pedido.nr_pedido
WHERE TO_CHAR(pedido.dt_pedido, 'YYYY-MM') >= '2020-01' AND TO_CHAR(pedido.dt_pedido, 'YYYY-MM') < '2021-01'
GROUP BY TO_CHAR(pedido.dt_pedido, 'YYYY-MM'));

SELECT *
FROM tabela_temporaria
WHERE ranque <= 3;

In [ ]:
-- Quais os 5 vendedores receberam mais comissão em 2019?

CREATE OR REPLACE VIEW tabela_temporaria AS
(SELECT 
	pedido.cd_vendedor,
	vendedor.nm_vendedor,
	ROUND((SUM(item_pedido.qnt_item_pedido * item_pedido.preco_item * (vendedor.percentual_comissao))/100)::numeric, 2) AS comissao_total,
	RANK () OVER (ORDER BY SUM(item_pedido.qnt_item_pedido * item_pedido.preco_item * (vendedor.percentual_comissao))/100 DESC) AS ranque
FROM item_pedido
LEFT JOIN pedido ON item_pedido.nr_pedido = pedido.nr_pedido
LEFT JOIN vendedor ON pedido.cd_vendedor = vendedor.cd_vendedor
GROUP BY pedido.cd_vendedor, vendedor.nm_vendedor);

SELECT *
FROM tabela_temporaria
WHERE ranque <= 5;

In [ ]:
-- Qual o produto mais vendido nos meses de julho e dezembro?

WITH tabela_temporaria AS
(SELECT 
	TO_CHAR(pedido.dt_pedido, 'MM') AS mes,
	item_pedido.cd_produto,
	SUM(item_pedido.qnt_item_pedido) AS quantidade_vendas,
	RANK () OVER (PARTITION BY TO_CHAR(pedido.dt_pedido, 'MM') ORDER BY SUM(item_pedido.qnt_item_pedido) DESC) AS ranque
FROM item_pedido
LEFT JOIN pedido ON item_pedido.nr_pedido = pedido.nr_pedido
GROUP BY TO_CHAR(pedido.dt_pedido, 'MM'), item_pedido.cd_produto
HAVING TO_CHAR(pedido.dt_pedido, 'MM') = '07' OR TO_CHAR(pedido.dt_pedido, 'MM') = '12')

SELECT *
FROM tabela_temporaria
WHERE ranque = 1;